<h1><span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#import" data-toc-modified-id="import-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>import</a></span></li><li><span><a href="#read/head" data-toc-modified-id="read/head-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>read/head</a></span></li><li><span><a href="#metrics" data-toc-modified-id="metrics-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>metrics</a></span></li><li><span><a href="#&quot;label&quot;" data-toc-modified-id="&quot;label&quot;-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>"label"</a></span></li><li><span><a href="#split" data-toc-modified-id="split-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>split</a></span></li><li><span><a href="#model" data-toc-modified-id="model-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>model</a></span></li><li><span><a href="#test" data-toc-modified-id="test-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>test</a></span></li><li><span><a href="#baseline" data-toc-modified-id="baseline-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>baseline</a></span></li></ul></div>

## import

In [1]:
!pip install sentence_transformers -q

In [2]:
import pandas as pd
from sklearn.metrics import f1_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier, Pool

## baseline

In [3]:
if __name__ == "__main__":
    df_train = pd.read_csv("data/train.csv")
    df_test = pd.read_csv("data/test.csv")

In [6]:
y_train=df_train["label"].map({'ai_answer': 1, 'hu_answer': 0})
y_train    

0       0
1       1
2       1
3       0
4       0
       ..
4814    1
4815    0
4816    0
4817    1
4818    1
Name: label, Length: 4819, dtype: int64

In [8]:
%%time
train_pool = Pool(
    data=df_train[['ans_text','q_title']],
    label=y_train,
    text_features=['ans_text','q_title']
)

model = CatBoostClassifier(iterations=500, learning_rate=0.1, eval_metric='F1') 

model.fit(
    train_pool,
   # eval_set=eval_pool,
    plot=True,
    verbose=True,
    #use_best_model=True,
    early_stopping_rounds=50,
    metric_period=10
)    

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.9077236	total: 234ms	remaining: 1m 56s
10:	learn: 0.9359401	total: 1.49s	remaining: 1m 6s
20:	learn: 0.9475000	total: 2.69s	remaining: 1m 1s
30:	learn: 0.9529559	total: 3.92s	remaining: 59.3s
40:	learn: 0.9558885	total: 5.11s	remaining: 57.3s
50:	learn: 0.9586880	total: 6.41s	remaining: 56.4s
60:	learn: 0.9620725	total: 7.61s	remaining: 54.8s
70:	learn: 0.9633009	total: 8.78s	remaining: 53s
80:	learn: 0.9672063	total: 9.96s	remaining: 51.5s
90:	learn: 0.9684647	total: 11.2s	remaining: 50.2s
100:	learn: 0.9686787	total: 12.3s	remaining: 48.8s
110:	learn: 0.9695084	total: 13.5s	remaining: 47.3s
120:	learn: 0.9724581	total: 14.6s	remaining: 45.6s
130:	learn: 0.9726595	total: 15.7s	remaining: 44.2s
140:	learn: 0.9747412	total: 16.8s	remaining: 42.8s
150:	learn: 0.9759934	total: 17.9s	remaining: 41.4s
160:	learn: 0.9772445	total: 19.1s	remaining: 40.2s
170:	learn: 0.9790934	total: 20.3s	remaining: 39s
180:	learn: 0.9809839	total: 21.5s	remaining: 37.8s
190:	learn: 0.9822314	tota

In [ ]:
y_pred=model.predict(df_test[['ans_text','q_title']])

df_test["label"] = y_pred
df_test["label"] = df_test["label"].map({1: 'ai_answer', 0: 'hu_answer'})
df_test[["line_id", "label"]].to_csv("data/submission.csv", sep=",", index=False)
